# Análisis de Correlación entre Limpieza de Aulas y Rendimiento Académico

Este notebook analiza la relación entre la limpieza de las aulas, la disponibilidad de materiales de limpieza y el rendimiento académico de los estudiantes mediante la generación y análisis de datos sintéticos.

## 1. Importar Librerías Necesarias

Importaremos las librerías necesarias para el análisis de datos, visualización y generación de datos sintéticos.

In [ ]:
# Librerías para manipulación de datos
import pandas as pd
import numpy as np

# Librerías para visualización
import matplotlib.pyplot as plt
import seaborn as sns

# Librerías para análisis estadístico
from scipy import stats
from scipy.stats import chi2_contingency, f_oneway

# Configuración de visualización
plt.style.use('default')
sns.set_palette("husl")
plt.rcParams['figure.figsize'] = (10, 6)

# Para reproducibilidad
np.random.seed(42)

## 2. Generar Conjunto de Datos Sintético

Crearemos un dataset sintético con 300 registros que incluya todas las variables solicitadas, asegurando que las notas sean mediocres cuando el nivel de aseo es regular o deficiente.

In [ ]:
# Definir parámetros del dataset
n_records = 300

# Generar IDs únicos para aulas y estudiantes
aulas = [f"{letra}-{str(num).zfill(3)}" for letra in ['A', 'B', 'C', 'D'] for num in range(101, 126)]
id_aula = np.random.choice(aulas, n_records)

# Generar IDs únicos para estudiantes
id_estudiante = [f"EST-{str(i).zfill(4)}" for i in range(1001, 1001 + n_records)]

# Generar niveles de aseo con distribución específica
nivel_aseo_opciones = ['Excelente', 'Bueno', 'Regular', 'Deficiente']
nivel_aseo_prob = [0.15, 0.35, 0.35, 0.15]  # Más casos en categorías intermedias
nivel_aseo = np.random.choice(nivel_aseo_opciones, n_records, p=nivel_aseo_prob)

# Generar disponibilidad de materiales
materiales_disponibles = np.random.choice([True, False], n_records, p=[0.7, 0.3])

# Generar notas finales basadas en el nivel de aseo
nota_final = []
for aseo in nivel_aseo:
    if aseo == 'Excelente':
        nota = np.random.normal(8.5, 1.0)  # Media alta, baja dispersión
    elif aseo == 'Bueno':
        nota = np.random.normal(7.0, 1.2)  # Media buena
    elif aseo == 'Regular':
        nota = np.random.normal(5.0, 1.0)  # Media mediocre
    else:  # Deficiente
        nota = np.random.normal(4.5, 1.2)  # Media baja
    
    # Asegurar que las notas estén entre 1 y 10
    nota = max(1, min(10, nota))
    nota_final.append(round(nota, 1))

# Crear el DataFrame
df = pd.DataFrame({
    'ID_Aula': id_aula,
    'ID_Estudiante': id_estudiante,
    'Nivel_Aseo': nivel_aseo,
    'Materiales_Disponibles': materiales_disponibles,
    'Nota_Final': nota_final
})

print(f"Dataset creado con {len(df)} registros")
print(f"Columnas: {list(df.columns)}")

: 

## 3. Explorar la Estructura de los Datos

Examinemos la estructura del dataset generado y verifiquemos la calidad de los datos.

In [ ]:
# Mostrar las primeras filas del dataset
print("Primeras 10 filas del dataset:")
print(df.head(10))
print("\n" + "="*50 + "\n")

# Información general del dataset
print("Información general del dataset:")
print(df.info())
print("\n" + "="*50 + "\n")

# Verificar valores nulos
print("Valores nulos por columna:")
print(df.isnull().sum())
print("\n" + "="*50 + "\n")

# Verificar duplicados
print(f"Registros duplicados: {df.duplicated().sum()}")
print("\n" + "="*50 + "\n")

# Distribución de variables categóricas
print("Distribución del Nivel de Aseo:")
print(df['Nivel_Aseo'].value_counts())
print("\nDistribución de Materiales Disponibles:")
print(df['Materiales_Disponibles'].value_counts())

## 4. Calcular Estadísticas Descriptivas

Calculemos estadísticas descriptivas para las variables numéricas, especialmente para la columna Nota_Final.

In [ ]:
# Estadísticas descriptivas generales
print("Estadísticas descriptivas de Nota_Final:")
print(df['Nota_Final'].describe())
print("\n" + "="*50 + "\n")

# Estadísticas adicionales
print("Estadísticas adicionales de Nota_Final:")
print(f"Media: {df['Nota_Final'].mean():.2f}")
print(f"Mediana: {df['Nota_Final'].median():.2f}")
print(f"Moda: {df['Nota_Final'].mode().iloc[0]:.1f}")
print(f"Desviación estándar: {df['Nota_Final'].std():.2f}")
print(f"Varianza: {df['Nota_Final'].var():.2f}")
print(f"Coeficiente de variación: {(df['Nota_Final'].std() / df['Nota_Final'].mean()) * 100:.2f}%")
print(f"Asimetría: {df['Nota_Final'].skew():.2f}")
print(f"Curtosis: {df['Nota_Final'].kurtosis():.2f}")
print("\n" + "="*50 + "\n")

# Rango de notas
print(f"Nota mínima: {df['Nota_Final'].min():.1f}")
print(f"Nota máxima: {df['Nota_Final'].max():.1f}")
print(f"Rango: {df['Nota_Final'].max() - df['Nota_Final'].min():.1f}")

## 5. Analizar Distribución de Notas por Nivel de Aseo

Agrupemos los datos por nivel de aseo y calculemos estadísticas descriptivas para cada grupo.

In [ ]:
# Agrupar por nivel de aseo y calcular estadísticas
print("Estadísticas descriptivas de Nota_Final por Nivel de Aseo:")
estadisticas_por_aseo = df.groupby('Nivel_Aseo')['Nota_Final'].agg([
    'count', 'mean', 'median', 'std', 'min', 'max'
]).round(2)
print(estadisticas_por_aseo)
print("\n" + "="*50 + "\n")

# Análisis por materiales disponibles
print("Estadísticas descriptivas de Nota_Final por Materiales Disponibles:")
estadisticas_por_materiales = df.groupby('Materiales_Disponibles')['Nota_Final'].agg([
    'count', 'mean', 'median', 'std', 'min', 'max'
]).round(2)
print(estadisticas_por_materiales)
print("\n" + "="*50 + "\n")

# Análisis cruzado: Nivel de aseo y materiales
print("Promedio de notas por Nivel de Aseo y Materiales Disponibles:")
tabla_cruzada = pd.crosstab(df['Nivel_Aseo'], df['Materiales_Disponibles'], 
                           values=df['Nota_Final'], aggfunc='mean').round(2)
print(tabla_cruzada)
print("\n" + "="*50 + "\n")

# Contar registros por combinación
print("Número de registros por Nivel de Aseo y Materiales Disponibles:")
conteo_cruzado = pd.crosstab(df['Nivel_Aseo'], df['Materiales_Disponibles'])
print(conteo_cruzado)

## 6. Crear Visualizaciones de la Distribución

Generemos diversos gráficos para visualizar la distribución de las notas según el nivel de aseo del aula.

In [ ]:
# Configurar el estilo de los gráficos
fig, axes = plt.subplots(2, 2, figsize=(15, 12))
fig.suptitle('Análisis de Distribución de Notas por Nivel de Aseo', fontsize=16, fontweight='bold')

# 1. Histograma general de notas
axes[0,0].hist(df['Nota_Final'], bins=20, alpha=0.7, color='skyblue', edgecolor='black')
axes[0,0].set_title('Distribución General de Notas Finales')
axes[0,0].set_xlabel('Nota Final')
axes[0,0].set_ylabel('Frecuencia')
axes[0,0].axvline(df['Nota_Final'].mean(), color='red', linestyle='--', 
                  label=f'Media: {df["Nota_Final"].mean():.2f}')
axes[0,0].legend()

# 2. Box plot por nivel de aseo
orden_aseo = ['Deficiente', 'Regular', 'Bueno', 'Excelente']
sns.boxplot(data=df, x='Nivel_Aseo', y='Nota_Final', order=orden_aseo, ax=axes[0,1])
axes[0,1].set_title('Distribución de Notas por Nivel de Aseo')
axes[0,1].set_xlabel('Nivel de Aseo')
axes[0,1].set_ylabel('Nota Final')
axes[0,1].tick_params(axis='x', rotation=45)

# 3. Gráfico de barras de promedios
promedios = df.groupby('Nivel_Aseo')['Nota_Final'].mean().reindex(orden_aseo)
colores = ['#ff7f7f', '#ffbf7f', '#7fbf7f', '#7f7fff']
bars = axes[1,0].bar(promedios.index, promedios.values, color=colores, alpha=0.8, edgecolor='black')
axes[1,0].set_title('Promedio de Notas por Nivel de Aseo')
axes[1,0].set_xlabel('Nivel de Aseo')
axes[1,0].set_ylabel('Promedio de Nota Final')
axes[1,0].tick_params(axis='x', rotation=45)

# Agregar valores en las barras
for bar, valor in zip(bars, promedios.values):
    axes[1,0].text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.1, 
                   f'{valor:.2f}', ha='center', va='bottom', fontweight='bold')

# 4. Violin plot
sns.violinplot(data=df, x='Nivel_Aseo', y='Nota_Final', order=orden_aseo, ax=axes[1,1])
axes[1,1].set_title('Distribución Detallada de Notas por Nivel de Aseo')
axes[1,1].set_xlabel('Nivel de Aseo')
axes[1,1].set_ylabel('Nota Final')
axes[1,1].tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.show()

In [ ]:
# Gráfico adicional: Análisis por materiales disponibles
fig, axes = plt.subplots(1, 2, figsize=(15, 6))
fig.suptitle('Análisis de Notas por Disponibilidad de Materiales', fontsize=16, fontweight='bold')

# Box plot por materiales disponibles
sns.boxplot(data=df, x='Materiales_Disponibles', y='Nota_Final', ax=axes[0])
axes[0].set_title('Distribución de Notas por Materiales Disponibles')
axes[0].set_xlabel('Materiales Disponibles')
axes[0].set_ylabel('Nota Final')

# Gráfico de barras agrupadas: Nivel de aseo y materiales
df_pivot = df.pivot_table(values='Nota_Final', index='Nivel_Aseo', 
                         columns='Materiales_Disponibles', aggfunc='mean')
df_pivot.reindex(orden_aseo).plot(kind='bar', ax=axes[1], 
                                 color=['#ff9999', '#66b3ff'], alpha=0.8)
axes[1].set_title('Promedio de Notas por Nivel de Aseo y Materiales')
axes[1].set_xlabel('Nivel de Aseo')
axes[1].set_ylabel('Promedio de Nota Final')
axes[1].legend(title='Materiales Disponibles', labels=['No', 'Sí'])
axes[1].tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.show()

## 7. Calcular Correlaciones entre Variables

Calculemos las correlaciones entre las variables numéricas y categóricas codificadas.

In [ ]:
# Crear una copia del dataframe para codificar variables categóricas
df_encoded = df.copy()

# Codificar variables categóricas
# Nivel de Aseo: ordinal encoding
nivel_aseo_map = {'Deficiente': 1, 'Regular': 2, 'Bueno': 3, 'Excelente': 4}
df_encoded['Nivel_Aseo_Num'] = df_encoded['Nivel_Aseo'].map(nivel_aseo_map)

# Materiales Disponibles: binary encoding
df_encoded['Materiales_Disponibles_Num'] = df_encoded['Materiales_Disponibles'].astype(int)

# Seleccionar solo variables numéricas para correlación
variables_numericas = ['Nivel_Aseo_Num', 'Materiales_Disponibles_Num', 'Nota_Final']
df_correlacion = df_encoded[variables_numericas]

# Calcular matriz de correlación
matriz_correlacion = df_correlacion.corr()

print("Matriz de Correlación:")
print(matriz_correlacion.round(3))
print("\n" + "="*50 + "\n")

# Correlaciones específicas con Nota_Final
print("Correlaciones con Nota_Final:")
correlaciones_nota = matriz_correlacion['Nota_Final'].sort_values(ascending=False)
print(correlaciones_nota.round(3))
print("\n" + "="*50 + "\n")

# Interpretación de correlaciones
corr_aseo_nota = matriz_correlacion.loc['Nivel_Aseo_Num', 'Nota_Final']
corr_materiales_nota = matriz_correlacion.loc['Materiales_Disponibles_Num', 'Nota_Final']

print("Interpretación de correlaciones:")
print(f"Correlación Nivel de Aseo - Nota Final: {corr_aseo_nota:.3f}")
if abs(corr_aseo_nota) >= 0.7:
    fuerza = "fuerte"
elif abs(corr_aseo_nota) >= 0.5:
    fuerza = "moderada"
elif abs(corr_aseo_nota) >= 0.3:
    fuerza = "débil"
else:
    fuerza = "muy débil"
print(f"  → Correlación {fuerza} {'positiva' if corr_aseo_nota > 0 else 'negativa'}")

print(f"\nCorrelación Materiales Disponibles - Nota Final: {corr_materiales_nota:.3f}")
if abs(corr_materiales_nota) >= 0.7:
    fuerza = "fuerte"
elif abs(corr_materiales_nota) >= 0.5:
    fuerza = "moderada"
elif abs(corr_materiales_nota) >= 0.3:
    fuerza = "débil"
else:
    fuerza = "muy débil"
print(f"  → Correlación {fuerza} {'positiva' if corr_materiales_nota > 0 else 'negativa'}")

## 8. Realizar Pruebas Estadísticas

Apliquemos pruebas estadísticas para determinar si existe una relación significativa entre las variables.

In [ ]:
# 1. ANOVA para comparar medias de notas entre niveles de aseo
grupos_aseo = []
for nivel in orden_aseo:
    grupos_aseo.append(df[df['Nivel_Aseo'] == nivel]['Nota_Final'])

# Realizar ANOVA
f_stat, p_value_anova = f_oneway(*grupos_aseo)

print("Prueba ANOVA - Diferencias entre niveles de aseo:")
print(f"Estadístico F: {f_stat:.4f}")
print(f"Valor p: {p_value_anova:.6f}")
print(f"Resultado: {'Hay diferencias significativas' if p_value_anova < 0.05 else 'No hay diferencias significativas'} (α = 0.05)")
print("\n" + "="*50 + "\n")

# 2. Prueba t para materiales disponibles
grupo_con_materiales = df[df['Materiales_Disponibles'] == True]['Nota_Final']
grupo_sin_materiales = df[df['Materiales_Disponibles'] == False]['Nota_Final']

t_stat, p_value_ttest = stats.ttest_ind(grupo_con_materiales, grupo_sin_materiales)

print("Prueba t - Diferencias por disponibilidad de materiales:")
print(f"Estadístico t: {t_stat:.4f}")
print(f"Valor p: {p_value_ttest:.6f}")
print(f"Resultado: {'Hay diferencias significativas' if p_value_ttest < 0.05 else 'No hay diferencias significativas'} (α = 0.05)")
print("\n" + "="*50 + "\n")

# 3. Prueba de normalidad de residuos
shapiro_stat, p_value_shapiro = stats.shapiro(df['Nota_Final'])

print("Prueba de Normalidad (Shapiro-Wilk) para Nota_Final:")
print(f"Estadístico W: {shapiro_stat:.4f}")
print(f"Valor p: {p_value_shapiro:.6f}")
print(f"Resultado: Los datos {'siguen' if p_value_shapiro > 0.05 else 'no siguen'} una distribución normal (α = 0.05)")
print("\n" + "="*50 + "\n")

# 4. Tabla de contingencia y Chi-cuadrado
# Crear categorías de notas para la prueba chi-cuadrado
df['Categoria_Nota'] = pd.cut(df['Nota_Final'], 
                             bins=[0, 4, 6, 8, 10], 
                             labels=['Baja', 'Media', 'Alta', 'Excelente'])

tabla_contingencia = pd.crosstab(df['Nivel_Aseo'], df['Categoria_Nota'])
print("Tabla de Contingencia - Nivel de Aseo vs Categoría de Nota:")
print(tabla_contingencia)

chi2, p_value_chi2, dof, expected = chi2_contingency(tabla_contingencia)

print(f"\nPrueba Chi-cuadrado:")
print(f"Estadístico Chi²: {chi2:.4f}")
print(f"Grados de libertad: {dof}")
print(f"Valor p: {p_value_chi2:.6f}")
print(f"Resultado: {'Hay asociación significativa' if p_value_chi2 < 0.05 else 'No hay asociación significativa'} (α = 0.05)")

## 9. Generar Gráficos de Correlación

Creemos mapas de calor y gráficos de dispersión para visualizar las correlaciones entre variables.

In [ ]:
# Crear figura con subplots para diferentes visualizaciones de correlación
fig, axes = plt.subplots(2, 2, figsize=(16, 12))
fig.suptitle('Análisis de Correlaciones y Relaciones entre Variables', fontsize=16, fontweight='bold')

# 1. Mapa de calor de correlaciones
sns.heatmap(matriz_correlacion, annot=True, cmap='RdBu_r', center=0, 
            square=True, fmt='.3f', cbar_kws={'shrink': 0.8}, ax=axes[0,0])
axes[0,0].set_title('Mapa de Calor - Matriz de Correlación')
axes[0,0].set_xlabel('')
axes[0,0].set_ylabel('')

# 2. Gráfico de dispersión: Nivel de Aseo vs Nota Final
colores_dispersión = {'Deficiente': '#ff7f7f', 'Regular': '#ffbf7f', 
                      'Bueno': '#7fbf7f', 'Excelente': '#7f7fff'}
for nivel in orden_aseo:
    subset = df[df['Nivel_Aseo'] == nivel]
    # Agregar un poco de ruido en x para mejor visualización
    x_jitter = np.random.normal(nivel_aseo_map[nivel], 0.1, len(subset))
    axes[0,1].scatter(x_jitter, subset['Nota_Final'], 
                     color=colores_dispersión[nivel], alpha=0.6, 
                     label=nivel, s=50)

axes[0,1].set_title('Dispersión: Nivel de Aseo vs Nota Final')
axes[0,1].set_xlabel('Nivel de Aseo (codificado)')
axes[0,1].set_ylabel('Nota Final')
axes[0,1].set_xticks([1, 2, 3, 4])
axes[0,1].set_xticklabels(['Deficiente', 'Regular', 'Bueno', 'Excelente'])
axes[0,1].legend()
axes[0,1].grid(True, alpha=0.3)

# 3. Box plot comparativo por materiales
df_materiales = df.copy()
df_materiales['Materiales_Texto'] = df_materiales['Materiales_Disponibles'].map({True: 'Con Materiales', False: 'Sin Materiales'})
sns.boxplot(data=df_materiales, x='Nivel_Aseo', y='Nota_Final', 
           hue='Materiales_Texto', order=orden_aseo, ax=axes[1,0])
axes[1,0].set_title('Distribución de Notas por Aseo y Materiales')
axes[1,0].set_xlabel('Nivel de Aseo')
axes[1,0].set_ylabel('Nota Final')
axes[1,0].tick_params(axis='x', rotation=45)
axes[1,0].legend(title='Disponibilidad')

# 4. Gráfico de líneas con promedios
promedios_por_grupo = df.groupby(['Nivel_Aseo', 'Materiales_Disponibles'])['Nota_Final'].mean().unstack()
promedios_por_grupo.reindex(orden_aseo).plot(kind='line', marker='o', 
                                            linewidth=2, markersize=8, ax=axes[1,1])
axes[1,1].set_title('Evolución de Promedios por Nivel de Aseo')
axes[1,1].set_xlabel('Nivel de Aseo')
axes[1,1].set_ylabel('Promedio de Nota Final')
axes[1,1].legend(title='Materiales Disponibles', labels=['No', 'Sí'])
axes[1,1].grid(True, alpha=0.3)
axes[1,1].tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.show()

## Conclusiones del Análisis

Basándome en el análisis realizado, aquí están las principales conclusiones sobre la relación entre la limpieza de las aulas y el rendimiento académico:

In [ ]:
# Generar resumen ejecutivo
print("RESUMEN EJECUTIVO - ANÁLISIS DE LIMPIEZA DE AULAS Y RENDIMIENTO ACADÉMICO")
print("="*80)

# Estadísticas clave
promedio_general = df['Nota_Final'].mean()
promedios_por_aseo = df.groupby('Nivel_Aseo')['Nota_Final'].mean()
diferencia_max_min = promedios_por_aseo.max() - promedios_por_aseo.min()

print(f"\n1. ESTADÍSTICAS GENERALES:")
print(f"   • Promedio general de notas: {promedio_general:.2f}/10")
print(f"   • Diferencia entre mejor y peor nivel de aseo: {diferencia_max_min:.2f} puntos")
print(f"   • Correlación nivel de aseo - notas: {corr_aseo_nota:.3f}")

print(f"\n2. PROMEDIOS POR NIVEL DE ASEO:")
for nivel in orden_aseo:
    promedio = promedios_por_aseo[nivel]
    print(f"   • {nivel:12}: {promedio:.2f}/10")

print(f"\n3. IMPACTO DE MATERIALES DE LIMPIEZA:")
promedio_con_materiales = df[df['Materiales_Disponibles']]['Nota_Final'].mean()
promedio_sin_materiales = df[~df['Materiales_Disponibles']]['Nota_Final'].mean()
diferencia_materiales = promedio_con_materiales - promedio_sin_materiales
print(f"   • Con materiales: {promedio_con_materiales:.2f}/10")
print(f"   • Sin materiales: {promedio_sin_materiales:.2f}/10")
print(f"   • Diferencia: {diferencia_materiales:.2f} puntos")

print(f"\n4. SIGNIFICANCIA ESTADÍSTICA:")
print(f"   • ANOVA (nivel de aseo): {'Significativo' if p_value_anova < 0.05 else 'No significativo'} (p = {p_value_anova:.4f})")
print(f"   • Prueba t (materiales): {'Significativo' if p_value_ttest < 0.05 else 'No significativo'} (p = {p_value_ttest:.4f})")

print(f"\n5. CONCLUSIONES PRINCIPALES:")
if corr_aseo_nota > 0.5:
    print(f"   • Existe una correlación FUERTE entre el nivel de aseo y las notas")
elif corr_aseo_nota > 0.3:
    print(f"   • Existe una correlación MODERADA entre el nivel de aseo y las notas")
else:
    print(f"   • La correlación entre el nivel de aseo y las notas es DÉBIL")

if diferencia_max_min > 2:
    print(f"   • La diferencia en rendimiento entre aulas limpias y sucias es CONSIDERABLE")
elif diferencia_max_min > 1:
    print(f"   • La diferencia en rendimiento entre aulas limpias y sucias es MODERADA")
else:
    print(f"   • La diferencia en rendimiento entre aulas limpias y sucias es PEQUEÑA")

if abs(diferencia_materiales) > 0.5:
    print(f"   • La disponibilidad de materiales de limpieza tiene un impacto NOTABLE")
else:
    print(f"   • La disponibilidad de materiales de limpieza tiene un impacto LIMITADO")

print(f"\n6. RECOMENDACIONES:")
print(f"   • Priorizar el mantenimiento de aulas en condición 'Excelente' o 'Buena'")
print(f"   • Asegurar disponibilidad constante de materiales de limpieza")
print(f"   • Implementar protocolos de limpieza regulares")
print(f"   • Monitorear periódicamente la correlación entre ambiente y rendimiento")

print("\n" + "="*80)